In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
import cv2
import os
from PIL import Image
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [12]:
# Path to the directory of images
image_directory = '/content/drive/MyDrive/Brain Tumor Detection/Datasets'

# Create a list of all the images in the directory
no_tumor_images = os.listdir(os.path.join(image_directory, 'no'))
yes_tumor_images = os.listdir(os.path.join(image_directory, 'yes'))

dataset = []
label = []
INPUT_SIZE = 64
# Process 'no' tumor images
for i, image_name in enumerate(no_tumor_images):
    if image_name.split('.')[1].lower() == 'jpg':  # Check if the file is a jpg image
        image_path = os.path.join(image_directory, 'no', image_name)  # Correct path here
        image = cv2.imread(image_path)  # Read the image

        # Only process if the image is successfully loaded
        if image is not None:
            image = Image.fromarray(image, 'RGB')
            image = image.resize((INPUT_SIZE,INPUT_SIZE))
            dataset.append(np.array(image))
            label.append(0)

# Process 'yes' tumor images
for i, image_name in enumerate(yes_tumor_images):
    if image_name.split('.')[1].lower() == 'jpg':  # Check if the file is a jpg image
        image_path = os.path.join(image_directory, 'yes', image_name)  # Correct path here
        image = cv2.imread(image_path)  # Read the image

        # Only process if the image is successfully loaded
        if image is not None:
            image = Image.fromarray(image, 'RGB')
            image = image.resize((INPUT_SIZE,INPUT_SIZE))
            dataset.append(np.array(image))
            label.append(1)

# Convert dataset and labels to numpy arrays
dataset = np.array(dataset)
label = np.array(label)

print(dataset)
print(label)

[[[[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  [[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  [[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  ...

  [[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  [[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  [[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]]


 [[[ 18  18  18]
   [ 18  18  18]
   [ 18  18  18]
   ...
   [ 18  18  18]
   [ 18  18  18]
   [ 18  18  18]]

  [[ 18  18  18]
   [ 18  18  18]
   [ 18  18  18]
   ...
   [ 18  18  18]
   [ 18  18  18]
   [ 18  18  18]]

  [[ 18  18  18]
   [ 18  18  18]
   [ 18  18  18]
   ...
   [ 18  18  18]
   [ 18  18  18]
   [ 18  18

In [13]:
print(len(dataset))
print(len(label))

3000
3000


In [14]:
dataset=np.array(dataset)
label = np.array(label)

In [15]:
X_train,X_test,y_train,y_test = train_test_split(dataset,label,test_size=0.2,random_state=0)

In [16]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2400, 64, 64, 3)
(600, 64, 64, 3)
(2400,)
(600,)


In [17]:

X_train = normalize(X_train,axis=1)
X_test = normalize(X_test,axis=1)
y_train = to_categorical(y_train,num_classes=2)
y_test = to_categorical(y_test,num_classes=2)

In [19]:
#Model building
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(32,(3,3),kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2)) #binary classification
model.add(Activation('softmax'))


#binary cross entropy dense = 1,use sigmoid  as activation
#cross entropy then dense = 2,softmax as activation

model.compile(loss='categorical_crossentropy',
              optimizer='adam'
              ,metrics=['accuracy'])
model.fit(X_train,y_train,batch_size=32,
          verbose=1,epochs=15,
          validation_data=(X_test,y_test),shuffle=False)
model.save('/content/drive/MyDrive/BrainTumorMainTrainCategorical15Binaryepochs.h5')



Epoch 1/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.6477 - loss: 0.6151 - val_accuracy: 0.7450 - val_loss: 0.5072
Epoch 2/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7813 - loss: 0.4902 - val_accuracy: 0.7967 - val_loss: 0.4535
Epoch 3/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8441 - loss: 0.4021 - val_accuracy: 0.8250 - val_loss: 0.3735
Epoch 4/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8580 - loss: 0.3353 - val_accuracy: 0.8483 - val_loss: 0.3238
Epoch 5/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8974 - loss: 0.2528 - val_accuracy: 0.8667 - val_loss: 0.2935
Epoch 6/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9170 - loss: 0.2102 - val_accuracy: 0.8983 - val_loss: 0.2281
Epoch 7/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9453 - loss: 0.1508 - val_accuracy: 0.9033 - val_loss: 0.2361
Epoch 8/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9615 - loss: 0.1140 - val_accuracy: 0.9417 - val_loss